In [1]:
import json
import random
from openai import AsyncOpenAI
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from tmdbv3api import Discover, Movie, TMDb
import os 
from dotenv import load_dotenv

load_dotenv()

tmdb = TMDb()
tmdb.api_key = os.environ.get('TMDB_API_KEY')
discover = Discover()
movie_api = Movie()

GENRE_MAP = {'action': 28,
 'adventure': 12,
 'animation': 16,
 'comedy': 35,
 'crime': 80,
 'documentary': 99,
 'drama': 18,
 'family': 10751,
 'fantasy': 14,
 'history': 36,
 'horror': 27,
 'music': 10402,
 'mystery': 9648,
 'romance': 10749,
 'science fiction': 878,
 'tv movie': 10770,
 'thriller': 53,
 'war': 10752,
 'western': 37}

In [2]:
class MovieTinderPlugin:
    @kernel_function(
        description="Step 1: Given genres JSON, return the top 2 movie titles, IDs & years per genre"
    )
    def fetch_options(self, genres_json: str) -> str:
        # 1) Parse the incoming genres list
        try:
            genres = json.loads(genres_json) if isinstance(genres_json, str) else genres_json
        except json.JSONDecodeError:
            genres = [genres_json]
        # 2) For each genre name, look up its ID and fetch top 2 by popularity
        result: dict[str, list[dict]] = {}
        for g in genres:
            gid = GENRE_MAP.get(g.lower())
            if not gid:
                continue
            raw = list(discover.discover_movies({
                "with_genres": gid,
                "sort_by":     "popularity.desc",
                "page":        1
            }))[:2]
            result[g.lower()] = [
                {
                    "id":           m.id,
                    "title":        m.title,
                    "release_year": (m.release_date or "").split("-")[0]
                }
                for m in raw
            ]
        return json.dumps(result)

    @kernel_function(
        description="Step 2: Given options JSON, pick one movie ID per genre"
    )
    def quiz_preferences(self, options_json: str) -> str:
        opts = json.loads(options_json)
        # auto‐pick the first movie for each genre for this prototype
        picks: list[int] = []
        for movies in opts:
            if isinstance(movies, list) and movies:
                picks.append(movies[0]["id"])
        return json.dumps(picks)

    @kernel_function(
        description="Step 3: Given picked IDs JSON, build a 25‐movie candidate pool per pick"
    )
    def build_candidates(self, picks_json: str) -> str:
        # 1) parse or coerce into Python list
        try:
            parsed = json.loads(picks_json)
        except json.JSONDecodeError:
            parsed = [int(picks_json)]
        # 2) flatten if it came in as dict
        if isinstance(parsed, dict):
            raw_ids: list[int] = []
            for v in parsed.values():
                raw_ids.extend(v if isinstance(v, list) else [v])
        elif isinstance(parsed, list):
            raw_ids = parsed
        else:
            raise ValueError(f"Invalid picks_input: {picks_json!r}")

        # 3) fetch top 25 popular movies for each pick
        pool: dict[str, list[dict]] = {}
        for pid in raw_ids:
            movies = list(discover.discover_movies({
                "sort_by": "popularity.desc",
                "page":    1
            }))[:25]
            pool[str(pid)] = [
                {
                    "id":           m.id,
                    "title":        m.title,
                    "release_year": (m.release_date or "").split("-")[0]
                }
                for m in movies
            ]
        return json.dumps(pool)

    @kernel_function(
        description="Step 4: From picks and candidates JSON, return 5 full movie recommendations"
    )
    def recommend(self, picks_json: str, candidates_json: str) -> str:
        picks = json.loads(picks_json)
        candidates = json.loads(candidates_json)
        # flatten & unique
        all_ids = []
        for cid_list in (candidates.values() if isinstance(candidates, dict) else []):
            for mid in cid_list:
                if mid not in all_ids:
                    all_ids.append(mid)
        # sample up to 5
        chosen = random.sample(all_ids, k=min(5, len(all_ids)))

        # fetch details for each
        recs: list[dict] = []
        for mid in chosen:
            m = movie_api.details(mid)
            recs.append({
                "id":           m.id,
                "title":        m.title,
                "release_year": (m.release_date or "").split("-")[0],
                "overview":     m.overview or "",
                "popularity":   m.popularity,
                "vote_average": m.vote_average,
            })
        return json.dumps(recs)


In [3]:
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

In [4]:
kernel = Kernel()

# 2) Register your MovieTinderPlugin under the namespace "MovieTinder"
#    (this makes its @kernel_function methods available as MovieTinder.fetch_options, etc.)
kernel.add_plugin(
    MovieTinderPlugin(),
    plugin_name="MovieTinder"
)

# 3) Build your ChatCompletionAgent as before
from semantic_kernel.agents import ChatCompletionAgent

agent = ChatCompletionAgent(
    service = chat_completion_service,
    plugins = [MovieTinderPlugin()],
    name    = "MovieAgent",
    instructions = """
You are MovieAgent. To answer any movie request, follow exactly:

1) CALL_FUNCTION: MovieTinder.fetch_options(genres='<JSON list of genres>')
2) Then CALL_FUNCTION: MovieTinder.quiz_preferences(options='<that JSON>')
3) Then CALL_FUNCTION: MovieTinder.build_candidates(picks='<that JSON>')
4) Finally CALL_FUNCTION: MovieTinder.recommend(picks='<step3 JSON>', candidates='<step4 JSON>')

Return only the final JSON array of 5 full movie recommendation objects
(each with id, title, release_year, overview, popularity, vote_average), and nothing else.
"""
)


In [7]:


thread: ChatHistoryAgentThread | None = None

print("🎬 Welcome to MovieTinder!\n")

# ─── STEP 1: Ask for genres ────────────────────────────────────────────────
print("Agent: Which genres are you in the mood for today? (e.g. Action, Comedy)")
user_msg = input()  # e.g. Action, Comedy

# Stream the agent’s first response (should CALL fetch_options)
async for resp in agent.invoke_stream(messages=user_msg, thread=thread):
    thread = resp.thread
    for item in resp.items:
        if hasattr(item, "text") and item.text:
            print(item.text, end="", flush=True)
        elif hasattr(item, "function_name"):
            print(f"\n→ Calling {item.function_name}({item.arguments})\n")
        elif hasattr(item, "result"):
            print(f"\n← Function result:\n{item.result}\n")
print()  # blank line

# ─── STEPS 2–5: quiz & recommend ───────────────────────────────────────────
while True:
    # The agent will have printed something like “1) Foo vs. Bar → ”
    choice = input()  # type “1” or “2” and press Enter
    async for resp in agent.invoke_stream(messages=choice, thread=thread):
        thread = resp.thread
        for item in resp.items:
            if hasattr(item, "text") and item.text:
                print(item.text, end="", flush=True)
            elif hasattr(item, "function_name"):
                print(f"\n→ Calling {item.function_name}({item.arguments})\n")
            elif hasattr(item, "result"):
                print(f"\n← Function result:\n{item.result}\n")
    print()  # blank line

    # Break once the agent has returned a JSON array of recommendations
    final_text = "".join(
        it.text for it in resp.items
        if hasattr(it, "text") and it.text
    ).strip()
    if final_text.startswith("[") and final_text.endswith("]"):
        break


🎬 Welcome to MovieTinder!

Agent: Which genres are you in the mood for today? (e.g. Action, Comedy)


Function failed. Error: Expecting value: line 1 column 1 (char 0)
Error invoking function MovieTinderPlugin-recommend: Expecting value: line 1 column 1 (char 0).
Traceback (most recent call last):
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/kernel.py", line 423, in _inner_auto_function_invoke_handler
    result = await context.function.invoke(context.kernel, context.arguments)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/functions/kernel_function.py", line 258, in invoke
    raise e
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/functions/kernel_function.py", line 250, in invoke
    await stack(function_context)
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-pac

Function failed. Error: Expecting value: line 1 column 1 (char 0)
Error invoking function MovieTinderPlugin-recommend: Expecting value: line 1 column 1 (char 0).
Traceback (most recent call last):
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/kernel.py", line 423, in _inner_auto_function_invoke_handler
    result = await context.function.invoke(context.kernel, context.arguments)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/functions/kernel_function.py", line 258, in invoke
    raise e
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-packages/semantic_kernel/functions/kernel_function.py", line 250, in invoke
    await stack(function_context)
  File "/Users/mendoza/Desktop/personal_dev/MSFT-Hackathon/mtind_venv/lib/python3.11/site-pac

NameError: name 'resp' is not defined